In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as f
from pathlib import Path
import numpy as np
import torchvision
from torch.utils.data import DataLoader, Dataset
import os
from torchsummary import torchsummary
from PIL import Image

In [2]:
# Local

ORIGINAL_IMAGE = 'E:/dataSets/NDI_images/20220725/20220725/Observed_Crop_200x200pix'
TARGET_IMAGE = 'E:/dataSets/NDI_images/20220725/20220725/Calculated_200x200/grayscale'

In [ ]:
# On Server

ORIGINAL_IMAGE = '/import/mqhome/duanct/dataSets/NDI_images/20220725/20220725/Observed_Crop_200x200pix'
TARGET_IMAGE = '/import/mqhome/duanct/dataSets/NDI_images/20220725/20220725/Calculated_200x200/grayscale'

In [3]:
class NDIDatasetContrastiveLearning(Dataset):
    def __init__(self):
        super(NDIDatasetContrastiveLearning, self).__init__()
        original_images = list(Path(ORIGINAL_IMAGE).glob('*.jpg'))
        origins, targets = [], []
        to_tensor_func = torchvision.transforms.ToTensor()
        for original_image in original_images:
            origins.append(to_tensor_func(Image.open(str(original_image))))
            targets.append(to_tensor_func(Image.open(str(Path.joinpath(Path(TARGET_IMAGE), original_image.name.split('_')[0] + '.jpg')))))
        random_index = np.random.permutation(len(origins))
        self.origins, self.targets = [], []
        for index in random_index:
            self.origins.append(origins[index])
            self.targets.append(targets[index])

    def __getitem__(self, idx):
        return self.origins[idx], self.targets[idx]

    def __len__(self):
        return len(self.origins)

In [25]:
from moco_model import MoCo


model = MoCo(torchvision.models.resnet18, dim=128, K=48)
device = torch.device('cuda:0')
criterion = nn.CrossEntropyLoss().cuda(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=1e-4)

In [11]:
# torchsummary.summary(model, input_size=[(1, 200, 200), (1, 200, 200)], batch_size=-1, device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 100, 100]           3,136
       BatchNorm2d-2         [-1, 64, 100, 100]             128
              ReLU-3         [-1, 64, 100, 100]               0
         MaxPool2d-4           [-1, 64, 50, 50]               0
            Conv2d-5           [-1, 64, 50, 50]          36,864
       BatchNorm2d-6           [-1, 64, 50, 50]             128
              ReLU-7           [-1, 64, 50, 50]               0
            Conv2d-8           [-1, 64, 50, 50]          36,864
       BatchNorm2d-9           [-1, 64, 50, 50]             128
             ReLU-10           [-1, 64, 50, 50]               0
       BasicBlock-11           [-1, 64, 50, 50]               0
           Conv2d-12           [-1, 64, 50, 50]          36,864
      BatchNorm2d-13           [-1, 64, 50, 50]             128
             ReLU-14           [-1, 64,

In [26]:
def train_moco(net, criterion, optimizer, epochs, device):
    train_iter = DataLoader(NDIDatasetContrastiveLearning(), batch_size=8, shuffle=True, drop_last=True)
    model.cuda(device)
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for origin, target in train_iter:
            origin, target = origin.cuda(device), target.cuda(device)
            output, labels = net(origin, target)
            loss = criterion(output, labels)
            total_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch + 1}, Loss {total_loss / len(train_iter)}')

In [27]:
train_moco(model, criterion, optimizer, 50, device)

Epoch 1, Loss 2.5646501675388813
Epoch 2, Loss 3.928589423497518
Epoch 3, Loss 3.739701747894287
Epoch 4, Loss 3.6790209213892617
Epoch 5, Loss 3.480889598528544
Epoch 6, Loss 3.2125312089920044
Epoch 7, Loss 3.0719483296076455
Epoch 8, Loss 2.9447721242904663
Epoch 9, Loss 2.9828642209370932
Epoch 10, Loss 2.783868392308553
Epoch 11, Loss 2.750709295272827
Epoch 12, Loss 2.775490125020345
Epoch 13, Loss 2.6246398289998374
Epoch 14, Loss 2.4811015526453652
Epoch 15, Loss 2.318331241607666
Epoch 16, Loss 2.3718751668930054
Epoch 17, Loss 2.2285926938056946
Epoch 18, Loss 2.08930104970932
Epoch 19, Loss 2.0435697038968406
Epoch 20, Loss 1.992450972398122
Epoch 21, Loss 2.0667046705881753
Epoch 22, Loss 1.8727173407872517
Epoch 23, Loss 1.8862409790356953
Epoch 24, Loss 1.8889602422714233
Epoch 25, Loss 1.77629421154658
Epoch 26, Loss 1.804925302664439
Epoch 27, Loss 1.809878985087077
Epoch 28, Loss 1.7513732711474101
Epoch 29, Loss 1.6811985969543457
Epoch 30, Loss 1.6697691082954407
Epo

In [7]:
def image_pair_matching(net, original_image, matching_image):
    net.cpu()
    net.eval()
    q = net.encoder_q(original_image)
    q = f.normalize(q, dim=1)
    k = net.encoder_k(matching_image)
    k = f.normalize(k, dim=1)
    logits = torch.mm(q, k.T)
    return logits

In [8]:
# Try to take the size of queue less than K, success

model.cpu()
original_images = list(Path(ORIGINAL_IMAGE).glob('*.jpg'))
target_images = list(Path(TARGET_IMAGE).glob('*.jpg'))
to_tensor_func = torchvision.transforms.ToTensor()
original_image = np.random.choice(original_images)
original_tensor = to_tensor_func(Image.open(str(original_image))).unsqueeze(0)
target_images_candidates = np.random.choice(target_images, 40)
target_tensor = []
for i in range(20):
    target_tensor.append(to_tensor_func(Image.open(str(target_images_candidates[i]))).unsqueeze(0))
target_tensor.append(to_tensor_func(Image.open(str(Path.joinpath(Path(TARGET_IMAGE), original_image.name.split('_')[0] + '.jpg')))).unsqueeze(0))
for i in range(20, 40):
    target_tensor.append(to_tensor_func(Image.open(str(target_images_candidates[i]))).unsqueeze(0))
target_tensor = torch.cat(target_tensor, dim=0)
image_pair_matching(model, original_tensor, target_tensor)

tensor([[0.1602, 0.1168, 0.1395, 0.1161, 0.2010, 0.1194, 0.0940, 0.0536, 0.1289,
         0.1064, 0.1281, 0.1714, 0.1407, 0.1310, 0.1047, 0.1310, 0.1194, 0.2436,
         0.1281, 0.1409, 0.3409, 0.0951, 0.1625, 0.1391, 0.1456, 0.1388, 0.1216,
         0.2257, 0.1398, 0.2367, 0.1064, 0.2324, 0.1189, 0.1511, 0.1047, 0.1310,
         0.1395, 0.0883, 0.0883, 0.1884, 0.1624]], grad_fn=<MmBackward>)

In [22]:
# Try to take all matching images

model.cpu()
original_images = list(Path(ORIGINAL_IMAGE).glob('*.jpg'))
target_images = list(Path(TARGET_IMAGE).glob('*.jpg'))
to_tensor_func = torchvision.transforms.ToTensor()
# original_image = np.random.choice(original_images)
original_image = Path.joinpath(Path(ORIGINAL_IMAGE), '11_z2-z_37-324-1.jpg')
# original_tensor = to_tensor_func(Image.open(str(original_image))).unsqueeze(0)
target_tensor = []
for i in range(1, 55):
    target_tensor.append(to_tensor_func(Image.open(str(Path.joinpath(Path(TARGET_IMAGE), f'{i}.jpg')))).unsqueeze(0))
target_tensor = torch.cat(target_tensor, dim=0)
print(str(original_image.name).split('_')[0])

11


In [23]:
logits = image_pair_matching(model, original_tensor, target_tensor)
print(logits)
print(torch.argmax(logits))

tensor([[ 0.1920,  0.1278,  0.1921,  0.2050,  0.0899,  0.0436,  0.0808,  0.1061,
          0.0628,  0.0947,  0.3432,  0.1393,  0.0484,  0.0799,  0.2761,  0.0530,
          0.1924,  0.1385,  0.0994,  0.1848,  0.1390,  0.1348,  0.1220,  0.0869,
          0.1555,  0.1555,  0.1115,  0.0206, -0.0127,  0.1123,  0.1606,  0.1342,
          0.1079,  0.1792,  0.1006,  0.1422,  0.0881,  0.0963,  0.1304,  0.1509,
          0.0807,  0.0987,  0.0826,  0.0631,  0.0564,  0.1104,  0.1550,  0.1939,
          0.1384,  0.1638,  0.1184,  0.0410,  0.0410,  0.0894]],
       grad_fn=<MmBackward>)
tensor(10)
